In [1]:
import numpy as np
from load import load_data
from convert import convert
from pmc2 import *
from brute_force import planes_def
from error_computation import *
from mc import *

In [2]:
#%matplotlib widget
#%matplotlib inline
%matplotlib qt
#%matplotlib gtk
import matplotlib.pyplot as plt
import matplotlib.font_manager
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
plt.rcParams.update({
    'pgf.rcfonts': False,
})

plt.rcParams['font.family'] = 'serif'
#from matplotlib.backends.backend_pgf import FigureCanvasPgf
#matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)

In [3]:
alf_inter = 1
alf_plane = 0.5

# Input

In [20]:
data = load_data(13)
d = convert(data)
P = Plane_HB(data)
#print(P.sol)
#Err,Err_P1,Err_P2 = error_computation(P,P)
#print(Err)

2.058780900891726


In [21]:
eq_pts_P_near = np.array([get_plane_normal_6_cycle(P,0),get_plane_normal_6_cycle(P,1),get_plane_normal_6_cycle(P,2)])
pts_P_near = np.linalg.solve(eq_pts_P_near,np.ones((3,1)))

In [22]:
sig_pts = P.pts #miss pts_P2
pts_with_largest_coord = sig_pts[:,np.argmax(np.max(sig_pts, axis=0))]
dist = np.dot(np.ones(3)/np.sqrt(3),pts_with_largest_coord)
#print(dist)

In [23]:
pts_P_far = p_plane_intersection_6(P,dist)

In [24]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
ax.plot(*P.pts,'k.')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
ax.plot(*pts_P_near,'ko',alpha=alf_inter)
#ax.plot(*pts_P1_far,'bo',alpha=alf_inter)
def get_verts(coord):
    return [list(zip(*cord))]

## Draw pyramid

In [25]:
for i in range(6):
    cord = np.zeros((3,3))
    cord[:,0] = pts_P_near[:,0]
    cord[:,1] = pts_P_far[:,i]
    cord[:,2] = pts_P_far[:,(i+1)%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=alf_plane)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)

In [26]:
fig.show()
#fig.savefig('3D.pdf')

# 2D view

In [27]:
new_base = np.zeros((3,3))

new_z = np.array([1,1,1])
new_y = np.array([-1,-1,2])
new_x = np.array([-1,1,0])
new_base[:,0] = new_x /np.linalg.norm(new_x )
new_base[:,1] = new_y /np.linalg.norm(new_y )
new_base[:,2] = new_z /np.linalg.norm(new_z )
new_base_transform = np.linalg.inv(new_base)

In [28]:
#new_sig = new_base_transform @ sig_pts
#new_transi = new_base_transform @ pts_transisiton_P12
new_P2 = new_base_transform @ pts_P_near
new_far_P1 = new_base_transform @ pts_P_far

In [29]:
#transi_dist_min = np.min(new_transi[2,:])
#transi_dist_max = np.max(new_transi[2,:])
#transi_dist_mid = (transi_dist_min+transi_dist_max)/2
#near_inter = p_plane_intersection_6(P2,transi_dist_min*0.4)
#near_transP2 = p_plane_intersection_6(P2,transi_dist_mid*1)
#near_transP1 = p_plane_intersection_6(P1,transi_dist_mid*1)
#new_near = new_base_transform @ near_inter
#new_transP2 = new_base_transform @ near_transP2
#new_transP1 = new_base_transform @ near_transP1

In [30]:
#transi_dist_max = np.max(new_transi[2,:])
#t = 0.5
#dist = transi_dist_max*t + transi_dist_min*(1-t)
#offset = 0 if np.allclose(new_transi[0,0],transi_dist_max) else 2
#offset = 0 if p_trans[0]>p_trans[1] else 2
#middle_inter = p_plane_intersection_12(P1,P2,dist,offset)
#new_middle = new_base_transform @ middle_inter

In [31]:
o_sig = d['o'][:,:3].transpose()
o_pts = np.zeros(o_sig.shape)
o_pts[0,:] = o_sig[1,:]
o_pts[1,:] = o_sig[2,:]
o_pts[2,:] = o_sig[0,:]
new_o_pts = new_base_transform @ o_pts
print(o_pts)

[[31.6  25.1  12.1  49.4  70.5  62.1  46.85]
 [ 5.    7.    9.   15.   20.   22.   10.  ]
 [48.3  52.9  63.9  70.6  77.5  83.9  88.14]]


In [44]:
p = np.unique(d['o'][:,3])
pts_tt1p = p_plane_intersection_6(P,p[0]*sqrt(3))
pts_tt2p = p_plane_intersection_6(P,p[1]*sqrt(3))
pts_tt3p = p_plane_intersection_6(P,p[3]*sqrt(3))
pts_tt1 = p_plane_intersection_6(P,p[2]*sqrt(3))

new_tt1p = new_base_transform @ pts_tt1p
new_tt2p = new_base_transform @ pts_tt2p
new_tt3p = new_base_transform @ pts_tt3p
new_tt1 = new_base_transform @ pts_tt1

In [49]:
plt.close()
#plt.axis('equal')
p1, = plt.plot(new_o_pts[0,:3],new_o_pts[1,:3],'r*', label='p={} MPa'.format(p[0]))
p2, = plt.plot(new_o_pts[0,3],new_o_pts[1,3],'b.',label='p={} MPa'.format(p[1]))
p3, = plt.plot(new_o_pts[0,4:6],new_o_pts[1,4:6],'gp',label='p={} MPa'.format(p[3]))
p4, = plt.plot(new_o_pts[0,6],new_o_pts[1,6],'yX',label='p={} MPa'.format(p[2]))
#plt.fill(new_P12[0,:],new_P12[1,:],edgecolor='k',fill=False)
plt.fill(new_far_P1[0,:],new_far_P1[1,:],edgecolor='k',fill=False)
plt.fill(new_P2[0,:],new_P2[1,:],edgecolor='k',fill=False)
#plt.fill(new_middle[0,:],new_middle[1,:],edgecolor='k',fill=False,label='12 sided - P2 and P1')
plt.fill(new_tt1p[0,:],new_tt1p[1,:],edgecolor='r',fill=False)
plt.fill(new_tt2p[0,:],new_tt2p[1,:],edgecolor='b',fill=False)
plt.fill(new_tt3p[0,:],new_tt3p[1,:],edgecolor='g',fill=False)
plt.fill(new_tt1[0,:],new_tt1[1,:],edgecolor='y',fill=False)
plt.legend(handles=[p1, p2, p3, p4])

In [50]:
x = np.linspace(0,100)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')

x = np.linspace(-110,0)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')

In [51]:
plt.xlabel('[MPa]')
plt.ylabel('[MPa]')
plt.axis('equal')
#plt.axis('off')
plt.show()

In [26]:
plt.close()